In [1]:
import numpy as np
import time

import syft as sy
import torch as th
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torch.autograd as autograd
import numpy as np

syft = sy 

hook = sy.TorchHook(th)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")
james = sy.VirtualWorker(hook, id="james")
crypto_provider = james
torch = th

from syft.serde.msgpack import serde

In [2]:
λ = 6 #6 #63 # security paramater
n = 8 #8 #32 # nb of bits of the algebric field

In [3]:
# PRG
def G(seed):
    #OK
    assert len(seed) == λ
    th.manual_seed(Convert(seed))
    return th.randint(2, size=(2*(λ + 1),), dtype=th.uint8)

#bit_pow_lambda = th.flip(2 ** th.arange(λ), (0,)).to(th.uint8)
def Convert(bits):
    bit_pow_lambda = th.flip(2 ** th.arange(λ), (0,)).to(th.uint8)
    return bits.dot(bit_pow_lambda)

def Array(*shape):
    #FIXED SHAPE
    return th.empty(shape, dtype=th.uint8)

#bit_pow_n = th.flip(2 ** th.arange(n), (0,))
def bit_decomposition(x):
    bit_pow_n = th.flip(2 ** th.arange(n), (0,))
    return ((x & bit_pow_n) > 0).to(th.int8)

def randbit(size):
    #FIXED SHAPE
    return th.randint(2, size=size)
    
def concat(*args, **kwargs):
    return th.cat(args, **kwargs)

def split(x, idx):
    return th.split(x, idx)
    
#one = th.tensor([1], dtype=th.uint8)
def TruthTableDPF(s, α_i):
    one = th.tensor([1], dtype=th.uint8)
    Table = th.zeros((2, λ+1), dtype=th.uint8)
    Table[α_i] = concat(s, one)
    return Table.flatten()

In [4]:
@sy.func2plan()
def GenDPF(alpha, beta):
    α = bit_decomposition(alpha)
    s, t, CW = Array(n+1, 2, λ), Array(n+1, 2), Array(n, 2*(λ+1))
    s[0] = randbit(size=(2, λ))
    t[0] = th.tensor([0, 1], dtype=torch.uint8)
    for i in range(0, n):
        # Re-use useless randomness
        sL_0, _, sR_0, _ = split(G(s[i, 0]), [λ, 1, λ, 1])
        sL_1, _, sR_1, _ = split(G(s[i, 1]), [λ, 1, λ, 1])
        s_rand = (sL_0 ^ sL_1) * α[i] + (sR_0 ^ sR_1) * (1 - α[i])
        
        cw_i = TruthTableDPF(s_rand, α[i])
        CW[i] = cw_i ^ G(s[i, 0]) ^ G(s[i, 1])
        
        for b in (0, 1):
            τ = G(s[i, b]) ^ (t[i, b] * CW[i])
            τ = τ.reshape(2, λ+1)
            s[i+1, b], t[i+1, b] = split(τ[𝛼[i]], [λ, 1])
        
    CW_n = (-1)**t[n, 1]*(beta.to(th.uint8) - Convert(s[n, 0]) + Convert(s[n, 1]))
    
    return s[0, 0], s[0, 1], CW, CW_n
        

In [5]:
@sy.func2plan()
def EvalDPF(b, x, *k_b):
    x = bit_decomposition(x)
    s, t = Array(n+1, λ), Array(n+1, 1)
    z = k_b[0] * 1
    s[0] = k_b[0]
    CW =  k_b[1:]
    t[0] = b
    for i in range(0, n):
        τ = G(s[i]) ^ (t[i]*CW[i])
        τ = τ.reshape(2, λ+1)
        s[i+1], t[i+1] = split(τ[x[i]], [λ, 1])
    return (-1)**b * (Convert(s[n]) + t[n]*CW[n])
        

In [6]:
beta = th.IntTensor([2])
alpha, = th.randint(0, 2**n, (1,))

GenDPF.build(alpha, beta)
res = GenDPF(alpha, beta)
s_00, s_01, CW, CW_n = res
k = [
    (s_00, *CW, CW_n),
    (s_01, *CW, CW_n)
]

In [7]:
EvalDPF.build(th.IntTensor([0]), alpha, *k[0])

In [8]:
serde_plan_simplified = serde._simplify(bob, GenDPF)
GenDPF_back = serde._detail(bob, serde_plan_simplified)

serde_plan_simplified = serde._simplify(bob, EvalDPF)
EvalDPF_back = serde._detail(bob, serde_plan_simplified)

In [9]:
from syft.messaging.message import Operation

In [10]:
beta = th.IntTensor([2])
alpha, = th.randint(0, 2**n, (1,))

In [11]:
res = GenDPF_back(alpha, beta)
s_00, s_01, CW, CW_n = res
k = [
    (s_00, *CW, CW_n),
    (s_01, *CW, CW_n)
]

In [12]:
(EvalDPF(th.IntTensor([0]), alpha, *k[0]) + EvalDPF(th.IntTensor([1]), alpha, *k[1]))

tensor([2], dtype=torch.int32)

In [13]:
(EvalDPF_back(th.IntTensor([0]), alpha+1, *k[0]) + EvalDPF_back(th.IntTensor([1]), alpha+1, *k[1]))

tensor([0], dtype=torch.int32)

In [14]:
def test_DPF():
    t = time.time()
    n_alpha, n_x = 10, 10
    beta = th.IntTensor([2])
    for alpha in th.randint(0, 2**n, (n_alpha, )):
        s_00, s_01, CW, CW_n = GenDPF(alpha, beta)
        k = [
            (s_00, *CW, CW_n),
            (s_01, *CW, CW_n)
        ]
        t2 = time.time()
        for x in th.randint(0, 2**n, (n_x,)):
            y0 = EvalDPF(th.IntTensor([0]), x, *k[0])
            y1 = EvalDPF(th.IntTensor([1]), x, *k[1])
            
            #print(y0+y1)
            if x == alpha:
                assert y0+y1 == beta
            else:
                assert y0+y1 == 0
              
        #print((EvalDPF(0, k[0], alpha) + EvalDPF(1, k[1], alpha)))
        assert (EvalDPF(0, alpha, *k[0]) + EvalDPF(1, alpha, *k[1])) == beta
        
        print(round(1000*(time.time() - t2)/((n_x+1) * 2), 1), 'ms / Eval')
    print(round(1000*(time.time() - t)/(n_alpha*n_x), 1), 'ms')
                
test_DPF()

4.3 ms / Eval
4.1 ms / Eval
4.3 ms / Eval
4.0 ms / Eval
4.2 ms / Eval
4.0 ms / Eval
4.2 ms / Eval
4.8 ms / Eval
5.2 ms / Eval
4.0 ms / Eval
12.1 ms


In [31]:
@sy.func2plan(args_shape=[(1,)])
def serde_plan(x):
    y = th.Tensor([-1])
    z = x + y
    return x

serde_plan_simplified = serde._simplify(bob, serde_plan)
serde_plan_detailed = serde._detail(bob, serde_plan_simplified)

for op in serde_plan_detailed.operations:
    print(op)

t = th.tensor([1.0])
expected = serde_plan_detailed(t)
actual = serde_plan_detailed(t)
assert actual == expected

print(actual)

('__add__', tensor([0.]), (tensor([-1.]),), {})


ValueError: 19707611155 is not in list

In [4]:
@sy.func2plan(args_shape=[(1,)])
def foo(x):
    th.manual_seed(14)
    y = th.randint(2, size=(1,), dtype=th.uint8)
    for i in range(0, 10):
        y = y + 1
    return y

serde_plan_simplified = serde._simplify(bob, foo)
serde_plan_detailed = serde._detail(bob, serde_plan_simplified)

t = th.tensor([1.0])
expected = foo(t)
actual = foo(t)
print(actual)
print(expected)
assert actual == expected

('torch.manual_seed', None, (14,), {})
('torch.randint', None, (2,), {'size': (1,), 'dtype': torch.uint8})
('__add__', tensor([1], dtype=torch.uint8), (1,), {})
('__add__', tensor([2], dtype=torch.uint8), (1,), {})
('__add__', tensor([3], dtype=torch.uint8), (1,), {})
('__add__', tensor([4], dtype=torch.uint8), (1,), {})
('__add__', tensor([5], dtype=torch.uint8), (1,), {})
('__add__', tensor([6], dtype=torch.uint8), (1,), {})
('__add__', tensor([7], dtype=torch.uint8), (1,), {})
('__add__', tensor([8], dtype=torch.uint8), (1,), {})
('__add__', tensor([9], dtype=torch.uint8), (1,), {})
('__add__', tensor([10], dtype=torch.uint8), (1,), {})
tensor([11], dtype=torch.uint8)
tensor([11], dtype=torch.uint8)


In [6]:
th.flip(2 ** th.arange(6), (0,)).to(th.uint8)

tensor([32, 16,  8,  4,  2,  1], dtype=torch.uint8)

In [9]:
kw = {'note_type': 'input'}
kw.get('note_type') == 'input'

True

In [4]:
serde_plan.find_placeholders('#input')

[PlaceHolder[Tags:#1 #input-0],
 PlaceHolder[Tags:#input-2 #3],
 PlaceHolder[Tags:#input-1 #5]]

In [2]:
t = th.tensor([3.0])
t.add

<bound method _TensorBase.add of tensor([3.])>

In [2]:
@sy.func2plan(args_shape=[(1,)], state=(th.tensor([3.0]),))
def plan(data, state):
    (bias,) = state.read()
    r = data * bias
    return r

In [3]:
plan.operations

[(Operation (('__mul__', PlaceHolder[Tags:#2 #input-0], (PlaceHolder[Tags:#state #1]>tensor([3.]),), {}), PlaceHolder[Tags:#output-0 #3]))]

In [1]:
@sy.func2plan(args_shape=[(1,)], state=(th.tensor([3.0]),))
def plan(data, state):
    (bias,) = state.read()
    r = data * bias
    return r

plan_pointer = plan.send(james)

print('---')
# Fetch plan
fetched_plan = plan_pointer.owner.fetch_plan(plan_pointer.id_at_location, james, copy=True)

print(fetched_plan.operations)


# Execute the fetch plan
x = th.tensor([-1.0])
print(fetched_plan)
result1 = fetched_plan(x)



NameError: name 'sy' is not defined

In [2]:
@sy.func2plan(args_shape=[(1,), (1,), (1,)])
def serde_plan(x, y, z):
    a = x + x  # 2
    b = x + z  # 4
    c = y + z  # 5
    return c, b, a  # 5, 4, 2

==
(tensor([0.]), tensor([0.]), tensor([0.]))
[12263725968, 83111581535, 79722608224]
('__add__', tensor([0.]), (tensor([0.]),), {})
('__add__', tensor([0.]), (tensor([0.]),), {})
('__add__', tensor([0.]), (tensor([0.]),), {})


In [4]:
@sy.func2plan(args_shape=[(1,)])
def serde_plan(x):
    x = x + x
    x = th.abs(x)
    return x

==
(tensor([0.]),)
[88027767395]
('__add__', tensor([0.]), (tensor([0.]),), {})
('torch.abs', None, (tensor([0.]),), {})


In [5]:
@sy.func2plan([th.Size((1, 3))])
def plan_double_abs(x):
    x = x.send(bob)
    x = x + x
    x = th.abs(x)
    return x

==
(tensor([[0., 0., 0.]]),)
[48698700407]
('__add__', (Wrapper)>[PointerTensor | me:15449418015 -> bob:48698700407], ((Wrapper)>[PointerTensor | me:15449418015 -> bob:48698700407],), {})


ValueError: 15449418015 is not in list

In [6]:
t = th.tensor([1.])
t.tag('#tags', '#lkjd')
t

tensor([1.])
	Tags: #tags #lkjd 
	Shape: torch.Size([1])

In [35]:
plan.fc1.weight += 1

RuntimeError: a leaf Variable that requires grad has been used in an in-place operation.

In [3]:
plan.state.state_placeholders[0].child += 1

RuntimeError: a leaf Variable that requires grad has been used in an in-place operation.

In [18]:
print(plan.state.state_placeholders[0].child)
print(plan.fc1.weight)

tensor([[1.9511]], grad_fn=<AddBackward0>)
Parameter containing:
tensor([[0.9511]], requires_grad=True)


In [2]:
@sy.func2plan(state=(th.tensor([3]),))
def plan_2(data, state):
    (bias,) = state.read()
    return data - bias

@sy.func2plan()
def plan_1(data):
    return 2 * plan_2(data)

x = th.tensor(100)
print('===')
plan_1.build(x)


===
READ ONLY ONCE inside plan_1
Add inner PlaceHolder(#1, #state)>tensor([3])
^^ 12787972444
47857151136
12787972444
76686916671
build ('__sub__', PlaceHolder(#input, #2), (PlaceHolder(#inner, #1, #state)>tensor([3]),), {}) PlaceHolder(#3)
76686916671
98867659172
build ('__rmul__', PlaceHolder(#3), (2,), {}) PlaceHolder(#4, #output-0)


In [3]:
plan_1.state.state_placeholders

[PlaceHolder(#inner, #1, #state)>tensor([3])]

In [5]:
expected_res = th.tensor(198)

@sy.func2plan(args_shape=[(1,)], state=(th.tensor([3]),))
def plan_3(data, state):
    (bias,) = state.read()
    return data - bias

@sy.func2plan(args_shape=[(1,)], state=(th.tensor([2]),))
def plan_2(data, state):
    (bias,) = state.read()
    return plan_3(data) + bias

@sy.func2plan()
def plan_1(data):
    return 2 * plan_2(data)

x = th.tensor(100)
print('===')
plan_1.build(x)

print(plan_1(x))
print(expected_res)

# Test locally
assert plan_1(x) == expected_res


print('***')
# Test remote
x_ptr = x.send(alice)
plan_1_ptr = plan_1.send(alice)

res = plan_1_ptr(x_ptr)
assert res.get() == expected_res

READ ONLY ONCE inside plan_3
49907160060
12904120921
90430603794
build ('__sub__', PlaceHolder(#input, #2), (PlaceHolder(#1, #state)>tensor([3]),), {}) PlaceHolder(#3, #output-0)
READ ONLY ONCE inside plan_2
READ ONLY ONCE inside plan_2
Add inner PlaceHolder(#1, #state)>tensor([3])
^^ 12904120921
55210876371
12904120921
26202437841
build ('__sub__', PlaceHolder(#3, #input), (PlaceHolder(#2, #inner, #state)>tensor([3]),), {}) PlaceHolder(#4)
26202437841
13802084914
60721122638
build ('__add__', PlaceHolder(#4), (PlaceHolder(#1, #state)>tensor([2]),), {}) PlaceHolder(#5, #output-0)
===
READ ONLY ONCE inside plan_1
Add inner PlaceHolder(#1, #state)>tensor([2])
^^ 13802084914
Add inner PlaceHolder(#2, #inner, #state)>tensor([3])
^^ 12904120921
READ ONLY ONCE inside plan_1
Add inner PlaceHolder(#1, #state)>tensor([3])
^^ 12904120921
21050643071
12904120921
95609084897
build ('__sub__', PlaceHolder(#input, #4), (PlaceHolder(#3, #inner, #state)>tensor([3]),), {}) PlaceHolder(#5)
95609084897
1

In [3]:
plan_1.state.state_placeholders

[PlaceHolder(#inner, #1, #state)>tensor([2]),
 PlaceHolder(#2, #inner, #state)>tensor([3])]

In [4]:
[p.copy() for p in plan_1.state.state_placeholders]

[PlaceHolder(#inner, #1, #state)>tensor([2]),
 PlaceHolder(#2, #inner, #state)>tensor([3])]

In [4]:


# Test locally
#assert plan_abs(x) == expected_res

# Test remote
x_ptr = x.send(alice)
plan_abs_ptr = plan_abs.send(alice)
print('---')
plan_abs_ptr(x_ptr)

res = plan_abs_ptr(x_ptr)
assert res.get() == expected_res

---

Instanciating inputs..

Running operations...

run cmd 0
('__rmul__', PlaceHolder(#2, #input)>tensor(100), (-2,), {})
PlaceHolder(-)>tensor(-200)
run cmd 1
('__add__', PlaceHolder(#3)>tensor(-200), (PlaceHolder(#state, #inner, #1)>tensor([1]),), {})
PlaceHolder(-)>tensor([-199])
run cmd 2
('abs', PlaceHolder(#4)>tensor([-199]), (), {})
PlaceHolder(-)>tensor([199])

Instanciating inputs..

Running operations...

run cmd 0
('__rmul__', PlaceHolder(#2, #input)>tensor(100), (-2,), {})
PlaceHolder(-)>tensor(-200)
run cmd 1
('__add__', PlaceHolder(#3)>tensor(-200), (PlaceHolder(#state, #inner, #1)>tensor([1]),), {})
PlaceHolder(-)>tensor([-199])
run cmd 2
('abs', PlaceHolder(#4)>tensor([-199]), (), {})
PlaceHolder(-)>tensor([199])


In [3]:
is_func2plan = True

if is_func2plan:

    @sy.func2plan(args_shape=[(1,)], state=(th.tensor([3.0]),))
    def plan(data, state):
        (bias,) = state.read()
        return data * bias

else:

    class Net(sy.Plan):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(1, 1)

        def forward(self, x):
            return self.fc1(x)

    plan = Net()
    plan.build(th.tensor([1.2]))

plan_pointer = plan.send(james)

# Fetch plan
fetched_plan = plan_pointer.owner.fetch_plan(plan_pointer.id_at_location, james)#, copy=True)

# Execute the fetch plan
x = th.tensor([-1.0])
result1 = fetched_plan(x)

print(result1)

? <Plan plan id:4030259375 owner:james built>

Instanciating inputs..

Running operations...

run cmd 0
('__mul__', PlaceHolder(#2, #input)>tensor([-1.]), (PlaceHolder(#state, #1)>tensor([3.]),), {})
PlaceHolder(-)>tensor([-3.])
tensor([-3.])


In [9]:
import re
m = re.search('#output-(.*)', '#output-7')
m.group(1)


'7'

In [14]:
for placeholder in serde_plan.placeholders.values():
    print(placeholder.tags)

{'#1', '#input'}
{'#2', '#output-0'}
{'#3'}


In [1]:
@sy.func2plan(args_shape=[(1,)])
def plan_abs(data):
    return data.abs()

assert isinstance(plan_abs.__str__(), str)
assert len(plan_abs.readable_plan) > 0
assert plan_abs.is_built

NameError: name 'sy' is not defined

In [3]:
@sy.func2plan(args_shape=[(1,)], state=(th.tensor([1.0]),))
def foo(x, state):
    (bias,) = state.read()
    x = x * 2
    return x + bias

assert isinstance(foo.__str__(), str)
assert len(foo.readable_plan) > 0
assert foo.is_built

t = th.tensor([1.0, 2])
x = foo(t)

assert (x == th.tensor([3.0, 5])).all()


Instanciating inputs..

Running operations...

run cmd 0
('__mul__', PlaceHolder(#2, #input)>tensor([1., 2.]), (2,), {})
PlaceHolder(#3)>tensor([2., 4.])
run cmd 1
('__add__', PlaceHolder(#3)>tensor([2., 4.]), (PlaceHolder(#1, #state)>tensor([1.]),), {})
PlaceHolder(#4)>tensor([3., 5.])


In [ ]:
from syft.serde.msgpack import serde

@sy.func2plan(args_shape=[(1,)])
def serde_plan(x):
    x = x + x
    y = x * 2
    return x

serde_plan_simplified = serde._simplify(bob, serde_plan)
serde_plan_detailed = serde._detail(bob, serde_plan_simplified)
t = th.tensor([2.])
expected = serde_plan(t)
print('expected: ', expected)
actual = serde_plan_detailed(t)
print('actual: ', actual)
assert actual == expected 


expected:  tensor([4.])

Instanciating inputs..

Running operations...

run cmd 0
('__add__', PlaceHolder(#1, #input)>tensor([2.]), (PlaceHolder(#1, #input)>tensor([2.]),), {})
PlaceHolder(-)>tensor([4.])
run cmd 1
('__mul__', PlaceHolder(#2)>tensor([4.]), (2,), {})
PlaceHolder(-)>tensor([8.])
actual:  tensor([8.])


AssertionError: 

In [2]:
def plan(func):
    def hooked_func(*args, **kwargs):
    
        syft.hook.trace, syft.hook.trace_inactive = True, True

        result = func(*args, **kwargs)

        syft.hook.trace, syft.hook.trace_inactive = False, False

        
        placeholders = {}
        var_count = [0]
        input_placeholders = []
        def add_placeholder(t, find_inputs=False):
            if t.id not in placeholders.keys():
                placeholder = sy.PlaceHolder(tags={f'#{var_count[0]+1}'})
                placeholders[t.id] = placeholder
                if find_inputs:
                    input_placeholders.append(placeholder)
                    placeholder.tags.add('#input')
                var_count[0] += 1
                    
            return placeholders[t.id]

        def replace_with_placeholders(obj, **kw):
            if isinstance(obj, (tuple, list)):
                r = [replace_with_placeholders(o, **kw) for o in obj]
                return type(obj)(r)
            elif isinstance(obj, dict):
                return {
                    key: replace_with_placeholders(value, **kw)
                    for key, value in obj.items()
                }
            elif isinstance(obj, th.Tensor):
                return add_placeholder(obj, **kw)
            elif isinstance(obj, (int, float, str, bool)):
                return obj
            elif obj is None:
                return None
            else:
                raise TypeError(f"Type {type(obj)} not supported in plans args/kwargs")


        new_logs = []
        for log in syft.trace_logs:
            req, resp = log
            req, resp = replace_with_placeholders(req, find_inputs=True), replace_with_placeholders(resp)
            new_logs.append((req, resp))
            
        print('==INPUTS==')
        print(input_placeholders)
        for log in new_logs:
            req, resp = log
            print('==========')
            print(req)
            print('-->', resp)
            
            
        print('\nInstanciating inputs to -0.7...\n')
        # Simulate instanciation
        for placeholder in input_placeholders:
            placeholder.instanciate(th.Tensor([-.7]))
            
        print('Running operations...\n')
        for i, log in enumerate(new_logs):
            print('run cmd', i)
            (cmd, self, args, kwargs), resp = log
            if self is None:
                r = eval(cmd)(*args, **kwargs)
            else:
                r = getattr(self, cmd)(*args, **kwargs)
            resp.instanciate(r.child)
            print(resp)
            

        syft.trace_logs = []
    
        return result
    
    return hooked_func

In [3]:
@plan
def foo(x, y):
    z = torch.add(x, y)
    r = z.abs()
    return r

In [4]:
x = th.Tensor([1.])
y = th.Tensor([2.])
m = foo(x, y)

==INPUTS==
[PlaceHolder(#1, #input), PlaceHolder(#2, #input)]
('torch.add', None, (PlaceHolder(#1, #input), PlaceHolder(#2, #input)), {})
--> PlaceHolder(#3)
('abs', PlaceHolder(#3), (), {})
--> PlaceHolder(#4)

Instanciating inputs to -0.7...

Running operations...

run cmd 0
PlaceHolder(#3)>tensor([-1.4000])
run cmd 1
PlaceHolder(#4)>tensor([1.4000])
